# Milestone 1: Evaluation metrics

In [1]:
import pandas as pd
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
housing_df = pd.read_csv("data/2023_Property_Tax_Assessment.csv")
housing_df = housing_df[['meters','garage','firepl','bsmt','bdevl','assess_2022']]
housing_df

,meters,garage,firepl,bsmt,bdevl,assess_2022
0,150.590,Y,Y,Y,N,382460
1,123.560,N,Y,N,N,280370
2,104.980,N,N,N,N,402000
3,66.611,N,N,N,N,3690
4,123.830,Y,Y,Y,Y,295910
...,...,...,...,...,...,...
35756,121.330,Y,Y,Y,Y,363000
35757,132.470,Y,Y,Y,N,355000
35758,121.330,Y,Y,Y,N,347000
35759,121.330,Y,Y,Y,Y,363000


In [3]:
train_df, test_df = train_test_split(housing_df, test_size=0.3, random_state=123)

In [4]:
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Lists of feature names
categorical_features = ['garage', 'firepl', 'bsmt', 'bdevl']
numeric_features = ['meters']
# Create the column transformer
preprocessor = make_column_transformer(
    (OneHotEncoder(), categorical_features),  # One-hot encode categorical columns (drop the first column to avoid redundancy)
    (StandardScaler(), numeric_features),  # Standardize numeric columns
)

# Show the preprocessor
preprocessor

ColumnTransformer(transformers=[('onehotencoder', OneHotEncoder(),
                                 ['garage', 'firepl', 'bsmt', 'bdevl']),
                                ('standardscaler', StandardScaler(),
                                 ['meters'])])

In [5]:
X_train = train_df.drop(columns=["assess_2022"])
X_test = test_df.drop(columns=["assess_2022"])
y_train = train_df["assess_2022"]
y_test = test_df["assess_2022"]

In [15]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import (
    cross_val_score,
    cross_validate
)
from sklearn.pipeline import make_pipeline

# The svc model pipeline
pipeline = make_pipeline(preprocessor, Ridge())

# The mean and std of the cross validated scores for all metrics as a dataframe
cross_val_results = pd.DataFrame(cross_validate(pipeline, X_train, y_train, cv=5, return_train_score=True)).agg(['mean', 'std']).round(3).T

# Show the train and validation scores
cross_val_results

,mean,std
fit_time,0.015,0.007
score_time,0.003,0.001
test_score,0.040,1.572
train_score,0.695,0.090


In [18]:
pipeline.fit(X_train, y_train)
pipeline.score(X_test, y_test)


0.34617887045802354

In [28]:
X_predict = pd.read_csv("data/2023_Property_Assessment_Predictions.csv").drop(columns='assess_2022')

In [38]:
y_predict = pipeline.predict(X_predict)
y_predict = pd.DataFrame(y_predict)
y_predict.columns = ['Predicted_Values']

In [39]:
pd.concat([X_predict,y_predict], axis = 1)

,meters,garage,firepl,bsmt,bdevl,Predicted_Values
0,174.23,Y,Y,Y,N,527136.172807
1,132.76,Y,N,Y,Y,451521.996667
2,90.82,Y,Y,N,Y,390750.919488
3,68.54,N,Y,N,N,224816.005694
4,221.30,Y,N,Y,Y,654274.738135
5,145.03,N,N,N,Y,419325.088911
6,102.96,N,N,Y,Y,328692.750689
7,164.28,Y,Y,N,N,498644.978037
8,142.79,N,Y,Y,N,400551.398300
9,115.94,Y,Y,Y,Y,453980.753156
